In [5]:
#https://stackoverflow.com/questions/51114736/xml-file-download-blocked-in-selenium-chromedriver
from requests import get
from bs4 import BeautifulSoup
import re
import os
from pathlib import Path
import platform
from xml.sax.saxutils import escape
import string

platform = platform.platform()
cwd = os.getcwd()
download_path = os.path.join(cwd,'Rebec Downloads')
#adjust this to fit your specific file structure 
parent = str(Path(cwd).parents[0]) 

In [2]:
base_rebec_url = 'http://www.ensaiosclinicos.gov.br/rg/?page='

if "Darwin" in platform:
    chrome_driver = os.path.join(parent, 'Drivers', 'chromedriver')
elif "Windows" in platform:
    chrome_driver = os.path.join(parent, 'Drivers', 'chromedriver.exe')
else:
    print("No OS/Chromedriver match. OS: {}".format(platform))

In [3]:
#Quick way to get max_page count
url = base_rebec_url + '1'
response = get(url, verify = False)
html = response.content

#gets parsed HTML
soup = BeautifulSoup(html, "html.parser")

number_of_pages = soup.find('div', {'class': 'pagination'})
max_page = number_of_pages.find_all('a')[-3].get_text()
print(max_page)

286


In [4]:
#getting the full trial count
home_page = get('http://www.ensaiosclinicos.gov.br')
hp_html = home_page.content
hp_soup = BeautifulSoup(hp_html, "html.parser")
tags = hp_soup.find_all('p')
for p in tags:
    if "ensaios publicados." in str(p):
        trial_count = int(p.find('span').text)
print(trial_count)

2858


In [5]:
#everything we need to run our selenium crawler
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep

page_list = [str(i) for i in range(2,int(max_page)+1)]
#after testing put this for full download: 'int(max_page)+1' instead of '5'

In [6]:
#this goes to the first search page, downloads the xml, then does that for every other page

chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : download_path,
        'profile.managed_default_content_settings.images':2,
        'disk-cache-size': 4096,
        'safebrowsing.enabled': 'false'}
chromeOptions.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(executable_path=chrome_driver, options=chromeOptions)
driver.get(base_rebec_url + '1')
#sleep(3)
driver.find_element_by_id("chk_toggle_selection").click()
driver.find_element_by_xpath("//input[@value='Download selected as OpenTrials XML format']").click()
#sleep(3)
for page in page_list:
    driver.find_element_by_link_text(page).click()
    driver.find_element_by_id("chk_toggle_selection").click()
    driver.find_element_by_xpath("//input[@value='Download selected as OpenTrials XML format']").click()
    #sleep(1)
sleep(5)
driver.quit()

 

In [3]:
#quick check to make sure we got the correct number of files
dl_counter = 0
for file in os.listdir(download_path):  
    if file.endswith('.xml'):
        dl_counter += 1

if dl_counter == max_page:  #4 is here just for testing, use 'max_page' variable for full implementation
    print("Full Download Successful")
else:
    print("Missing " + str(int(max_page)  - dl_counter) + " File(s)") #once again, replace the 4 here with 'max_page' in full implementation

Full Download Successful


In [7]:
#think if this can help? https://stackoverflow.com/questions/4324790/removing-control-characters-from-a-string-in-python
#or this https://stackoverflow.com/questions/17480656/remove-ascii-control-characters-from-text-file-python/43521569

import xmltodict
import json

#gets each trial and adds it to a list a json

rebec_trials = []
nonprintable = set([chr(i) for i in range(128)]).difference(string.printable)

for file in os.listdir(download_path):
    if file.endswith('.xml'):
        with open(os.path.join(download_path, file), encoding = 'utf-8') as xml:
            filtered_xml = "".join([b for b in xml.read() if b not in nonprintable])
            trials = xmltodict.parse(filtered_xml)
            for trial in trials['trials']['trial']:
                rebec_trials.append(json.dumps(trial))

In [19]:
#quick check to make sure things worked
print(rebec_trials[114])

{"@language": "en", "@status": "published", "@date_registration": "2016-09-12", "@created": "2016-04-18", "@updated": "2016-09-12", "trial_identification": {"trial_id": "RBR-83484y", "utrn_number": "U1111-1182-0732", "reg_name": "REBEC", "public_title": "Physical activity and healthy in elderly.", "acronym": null, "acronym_expansion": null, "scientific_title": "Resistance and concurrent training with blood flow restriction: hearat rate variability, systemic inflammation effects and neuromuscular adjustments in men and women.", "scientific_acronym": null, "scientific_acronym_expansion": null}, "sponsors_and_support": {"primary_sponsor": {"@country_code": "BR", "@type": "higher_education_institution", "name": "Faculdade de Educa\u00e7\u00e3o F\u00edsica, Laborat\u00f3rio de Fisiologia do Exerc\u00edcio, Universidade Estadual de Campinas", "address": "Faculdade de Educa\u00e7\u00e3o F\u00edsica\nAv Erico Ver\u00edssimo, 701 - Cidade Universit\u00e1ria Zeferino Vaz, Campinas, S\u00e3o Paul

In [17]:
#check
if len(rebec_trials) == trial_count:
    print("Success!: {} trials".format(trial_count))
else:
    print("Scrape Issue: {} on Rebec, {} in scrape".format(trial_count, len(rebec_trials)))

Success!: 2858 trials


In [15]:
#creates a csv with no headers and a single column with all trials in json in their own row.

from datetime import date
import csv

def rebec_csv():
    with open('rebec - ' + str(date.today()) + '.csv','w', newline = '')as rebec_csv:
        writer=csv.writer(rebec_csv)
        for val in rebec_trials:
            writer.writerow([val])

In [16]:
#rebec_csv()